In [136]:
from pprint import pprint
import jmespath
import requests
from bs4 import BeautifulSoup as BS4
from textwrap import dedent
import re

In [2]:
req = requests.get('https://www.selver.ee/')

In [3]:
soup = BS4(req.content, features='html.parser')

In [57]:
#esilehelt URL otsingu regex
url_class = re.compile(r'(item level0 nav.\w+.+\d)')
#Hinna ja toote otsingu regex
product_class = re.compile(r'(col-lg-3 \w+).+') 
product_price = re.compile(r'("price")')

In [5]:
#Leia kõik HTML Li tagilt mis hoiab URLi
find_tag = soup.find_all('li',{'class':class_to_find})

In [8]:
#leia URL'id ja lista toote nimekirja pikkus
get_urls = [i.find_next('a').get('href')+"/?limit=96" for i in find_tag]

In [12]:
get_urls

['https://www.selver.ee/minu-tooted/?limit=96',
 'https://www.selver.ee/uudistooted/?limit=96',
 'https://www.selver.ee/soodushinnaga-tooted/?limit=96',
 'https://www.selver.ee/eritooted/?limit=96',
 'https://www.selver.ee/hulgipakkumised/?limit=96',
 'https://www.selver.ee/puu-ja-koogiviljad/?limit=96',
 'https://www.selver.ee/liha-ja-kalatooted/?limit=96',
 'https://www.selver.ee/piimatooted-munad-void/?limit=96',
 'https://www.selver.ee/juustud/?limit=96',
 'https://www.selver.ee/leivad-saiad-kondiitritooted/?limit=96',
 'https://www.selver.ee/valmistoidud/?limit=96',
 'https://www.selver.ee/kuivained-hoidised/?limit=96',
 'https://www.selver.ee/maitseained-ja-puljongid/?limit=96',
 'https://www.selver.ee/kastmed-olid/?limit=96',
 'https://www.selver.ee/maiustused-kupsised-naksid/?limit=96',
 'https://www.selver.ee/kulmutatud-toidukaubad/?limit=96',
 'https://www.selver.ee/joogid/?limit=96',
 'https://www.selver.ee/lastekaubad/?limit=96',
 'https://www.selver.ee/lemmiklooma-kaubad/?

In [154]:
#Kogu html linkide kogust
product_html = requests.get(get_urls[8])
#puhasta ilusa seebiga
soup = BS4(product_html.content, features='html.parser')
#leia toote ja hinna kategoorijad li tagist
find_product_group = soup.find_all('li',{'class':product_class})
#leia hinnad
find_product_group_prices = soup.find_all('span',{'class':'price'})
#võta linki pealkirjad ehk toote nimed
find_product_group_titles = [i.find_next('a').get('title') for i in find_product_group]
#leia kõik hinnad
prices = [dedent(i.text.replace(u'\xa0',' ')) for i in find_product_group_prices if len(i) > 1]

In [155]:
#lisa kõik hinnad toote nimedele
dict_of_products = dict()
for t,f in zip(find_product_group_titles,prices):
    dict_of_products.update({t:f})

In [156]:
dict_of_products

{'Sulatatud juust, MEREVAIK, 200 g': '1,35 €  6,75 €/kg ',
 'Eesti juust 25,2% viilutatud, TRADITSIOONILINE EESTI JUUST, 500 g': '4,29 €  8,58 €/kg ',
 'Eesti Juust viilutatud, TRADITSIOONILINE EESTI JUUST, 350g': '1,99 €  5,69 €/kg ',
 'Hiirte Juust viilutatud, ESTOVER, 350 g': '3,29 €  9,40 €/kg ',
 'Juust Atleet Originaal, VALIO, 500 g': '2,79 €  5,58 €/kg ',
 'Eesti juust 25,2% riivitud, TRADITSIOONILINE EESTI JUUST, 200 g': '1,29 €  6,45 €/kg ',
 'Juust riivitud, SAAREMAA, 200 g': '1,49 €  7,45 €/kg ',
 'Eesti Juust riivitud, ESTOVER, 400g': '2,59 €  6,48 €/kg ',
 'Toorjuust Classic, PHILADELPHIA, 200 g': '2,19 €  10,95 €/kg ',
 'Sulatatud juust, VALIO VIOLA, 185 g': '1,39 €  7,51 €/kg ',
 'Viilujuust Atleet Originaal 26%, VALIO, 500 g': '4,39 €  8,78 €/kg ',
 'Vene juust 28,5% viilutatud, ESTOVER, 500 g': '3,99 €  7,98 €/kg ',
 'Sulatatud juust murulauguga, MEREVAIK, 200 g': '1,59 €  7,95 €/kg ',
 'Juustupulgad, PIK-NIK, 140 g': '2,79 €  19,93 €/kg ',
 'Riivjuust Cheddar, VALIO, 